In [ ]:
import pandas as pd
import re
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel # BERT 모델 및 토크나이저

In [ ]:
# CSV 파일에서 데이터 읽기
csv_file = r'C:\project_files\Multimodal_Auto _BGM_Addition_System\dataset_preprocessed\text\combined_emotion_dataset _neutral_x.csv'  
df = pd.read_csv(csv_file)

### 감정 비율 파악

In [ ]:
# 감정 레이블의 순서
emotion_order = ["anger", "disgust", "fear", "joy", "sad", "surprise"]

# 각 감정 레이블의 개수 출력
emotion_counts = df['감정'].value_counts()

# 지정된 emotion_order 순서에 따라 감정 개수 정렬
emotion_counts = emotion_counts.reindex(emotion_order, fill_value=0)

# 결과 출력
print("각 감정 레이블의 개수:")
print(emotion_counts)


### kobert 임베딩

In [ ]:
# 감정 라벨 매핑
emotion_label_mapping = {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5}

# 결측치가 있는 행을 삭제
df = df.dropna(subset=['문장', '감정'])

# 감정이 emotion_label_mapping에 존재하는 경우만 필터링
df = df[df['감정'].isin(emotion_label_mapping.keys())]

texts = df['문장'].tolist()
emotions = df['감정'].tolist()

# 텍스트와 감정 레이블이 잘 일치하는지 확인
if len(texts) != len(emotions):
    print(f"텍스트와 라벨의 길이가 다릅니다. 텍스트 개수: {len(texts)}, 라벨 개수: {len(emotions)}")

# 감정 라벨을 숫자로 변환
labels = [emotion_label_mapping[emotion] for emotion in emotions]

print(f"최종 텍스트 개수: {len(texts)}, 최종 라벨 개수: {len(labels)}")


In [ ]:
# 1. KoBERT 모델과 토크나이저 불러오기
model_name = 'monologg/kobert'  # Hugging Face에서 제공하는 KoBERT 모델 이름
tokenizer = BertTokenizer.from_pretrained(model_name)  # KoBERT 전용 토크나이저
model = BertModel.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 2. 텍스트 전처리 (특수문자 제거 함수 추가)
def clean_text(text):
    # 특수문자 및 비정상적인 문자를 제거 (한글, 영어, 숫자, 공백 외의 문자를 제거)
    text = re.sub(r'[^ㄱ-ㅎ가-힣a-zA-Z0-9\s]', '', text)
    return text.strip()

# 4. 텍스트 데이터 클리닝
texts = df['문장'].tolist()
texts = [clean_text(str(text)) for text in texts if isinstance(text, str)]

# 감정 데이터를 숫자로 변환 (emotion_label_mapping)
emotion_label_mapping = {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sad': 4, 'surprise': 5}
labels = [emotion_label_mapping[emotion] for emotion in df['감정'].tolist()]

# 5. 텍스트 전처리 함수 정의
def preprocess_texts(texts, tokenizer, max_len=128):
    inputs = tokenizer.batch_encode_plus(
        texts,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=max_len,
        add_special_tokens=True
    )
    return inputs

# 6. Dataset 클래스 정의
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label

# 7. 임베딩 추출 함수 정의
def get_embeddings(texts, model, tokenizer, batch_size=32):
    dataset = TextDataset(texts, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    all_embeddings = []

    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad():
        for batch_texts, _ in dataloader:
            inputs = preprocess_texts(batch_texts, tokenizer).to(device)
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state  # [CLS] 임베딩 추출
            cls_embeddings = embeddings[:, 0, :]  # [CLS] 토큰에 해당하는 임베딩
            all_embeddings.append(cls_embeddings.cpu().numpy())  # CPU로 이동 후 리스트에 저장

    all_embeddings = [embedding for batch in all_embeddings for embedding in batch]  # 2D 리스트를 1D로 변환
    return all_embeddings

# 8. 임베딩 추출
embeddings = get_embeddings(texts, model, tokenizer, batch_size=32)

# 9. 임베딩과 레이블의 개수를 맞추기 위해 길이 차이가 있을 경우, 마지막 값 삭제
if len(embeddings) != len(labels):
    print(f"임베딩 수: {len(embeddings)}, 라벨 수: {len(labels)}")
    if len(embeddings) > len(labels):
        embeddings = embeddings[:len(labels)]
    elif len(labels) > len(embeddings):
        labels = labels[:len(embeddings)]  # 문제가 되는 값 삭제

# 10. CSV로 저장할 데이터 준비 (문장과 임베딩을 결합)
embedding_df = pd.DataFrame(embeddings, columns=[f"embedding_{i}" for i in range(embeddings[0].shape[0])])  # 임베딩 열 생성
embedding_df['emotion_label'] = labels  # 감정 라벨 추가

# 11. CSV 파일로 저장
output_csv = 'text_embeddings_with_labels_new.csv'
embedding_df.to_csv(output_csv, index=False)
print(f"임베딩 데이터가 {output_csv}로 저장되었습니다.")
